In [21]:
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
# from non_stupid_sentence_splitter import SentenceSplitter
# I'm not sure what sentence splitter does, but it's way weird.
# SentenceWindowNodeParser is a lot more logical
# though it would be nice if it used Spacy or something smarter to split sentences
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache
from llama_index.core.node_parser import SentenceWindowNodeParser

In [14]:
text = """
The order plainly violates the Constitution and seeks to overturn crystal-clear Supreme Court
precedent. In those affirmations of the principle that anyone born in this country is 
automatically a citizen, race was the central issue — a fact that Trump and his advisers must
know. This history makes Trump’s order an act of performative racism that tells us, quite 
clearly, what kind of U.S.-born Americans he wants to exclude.

The saga begins before the Civil War with the Supreme Court’s infamous Dred Scott v. Sandford
decision in 1857, which denied citizenship to people of African descent even if they were not
enslaved. “A free negro of the African race, whose ancestors were brought to this country and
sold as slaves, is not a ‘citizen’ within the meaning of the Constitution of the United 
States,” the ruling held.

After the war, the race-based Dred Scott theory of citizenship was overturned by the very 
first sentence of the 14th Amendment, ratified in 1868: “All persons born or naturalized in 
the United States, and subject to the jurisdiction thereof, are citizens of the United States 
and of the State wherein they reside.” The whole point was race: Black people born here have 
the same status, and the same rights, as White people.
"""
# text = """
# New York City Council Members Want to Dial Down Sirens https://t.co/tQDZAaA2hh
# """


In [26]:
documents = [Document(text=text, metadata={
    "id_": "adsfsdf", 
    "asdfasdfasdfasd": "asdfasdfasdf", 
    "asdfasdfasdfads": "asdfasdfasdfasdfda",
    "asdfasdfasdfads3": "asdfasdfasdfasdfda",
    # "asdfasdfasdfads34": "asdfasdfasdfasdfda",
    # "asdfasdfasdfads5": "asdfasdfasdfasdfda",
    # "asdfasdfasdfads6": "asdfasdfasdfasdfda",
    # "asdfasdfasdfads7": "asdfasdfasdfasdfda"
    
})]


In [16]:
splitter = SentenceSplitter(chunk_size=50, chunk_overlap=10, include_metadata_in_chunksize=False)
splitter

SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x116cefc50>, id_func=<function default_id_func at 0x10cec6a20>, chunk_size=50, chunk_overlap=10, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?', include_metadata_in_chunksize=False)

In [17]:
"splitter thinks text has {} tokens; naively it has {} words".format(
    splitter._token_size(text),
    text.count(" ") + 1,    
)



'splitter thinks text has 274 tokens; naively it has 202 words'

In [18]:
nodes = splitter(documents)

In [19]:
# just for lookin'

print("N nodes: {}\n\n".format(len(nodes)))

# just for lookin'

for node in nodes[:10]:
    print(node.text)
    print(splitter._token_size(node.text))
    print("\n")

N nodes: 7


The order plainly violates the Constitution and seeks to overturn crystal-clear Supreme Court
precedent.
18


In those affirmations of the principle that anyone born in this country is 
automatically a citizen, race was the central issue — a fact that Trump and his advisers must
know.
37


This history makes Trump’s order an act of performative racism that tells us, quite 
clearly, what kind of U.S.-born Americans he wants to exclude.
34


The saga begins before the Civil War with the Supreme Court’s infamous Dred Scott v. Sandford
decision in 1857, which denied citizenship to people of African descent even if they were not
enslaved.
45


“A free negro of the African race, whose ancestors were brought to this country and
sold as slaves, is not a ‘citizen’ within the meaning of the Constitution of the United 
States,” the ruling held.

After the war,
48


After the war, the race-based Dred Scott theory of citizenship was overturned by the very 
first sentence of the 14th Am

In [13]:
nodes[0]

TextNode(id_='3fae55f7-8f06-46dc-9763-3d212a931e8a', embedding=None, metadata={'asdfsdf': 'adsfsdf', 'asdfasdfasdfasd': 'asdfasdfasdf', 'asdfasdfasdfads': 'asdfasdfasdfasdfda', 'asdfasdfasdfads3': 'asdfasdfasdfasdfda'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a30f9e94-8c2c-4cee-bd07-527c5aa9c5bd', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'asdfsdf': 'adsfsdf', 'asdfasdfasdfasd': 'asdfasdfasdf', 'asdfasdfasdfads': 'asdfasdfasdfasdfda', 'asdfasdfasdfads3': 'asdfasdfasdfasdfda'}, hash='3e2392b1e9200158f2c4b7bea1d1b15d75d698259eb07cd420fbdcf7ada209bf')}, metadata_template='{key}: {value}', metadata_separator='\n', text='New York City Council Members Want to Dial Down Sirens https://t.co/tQDZAaA2hh', mimetype='text/plain', start_char_idx=0, end_char_idx=78, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}')

In [75]:
from llama_index.core.schema import TransformComponent, NodeRelationship
windower = SentenceWindowNodeParser()
nodes = windower.get_nodes_from_documents(documents)
print(nodes[0].text)
nodes[0].relationships[NodeRelationship.NEXT].metadata["original_text"]

The order plainly violates the Constitution and seeks to overturn crystal-clear Supreme Court
precedent. 


'In those affirmations of the principle that anyone born in this country is \nautomatically a citizen, race was the central issue — a fact that Trump and his advisers must\nknow. '

In [62]:
# just for lookin'

print("N nodes: {}\n\n".format(len(nodes)))

# just for lookin'

for node in nodes[:10]:
    print(node.text)
    print(node.metadata["window"])
    print(splitter._token_size(node.text))
    print("\n")

N nodes: 6


The order plainly violates the Constitution and seeks to overturn crystal-clear Supreme Court
precedent. 
The order plainly violates the Constitution and seeks to overturn crystal-clear Supreme Court
precedent.  In those affirmations of the principle that anyone born in this country is 
automatically a citizen, race was the central issue — a fact that Trump and his advisers must
know.  This history makes Trump’s order an act of performative racism that tells us, quite 
clearly, what kind of U.S.-born Americans he wants to exclude.

 The saga begins before the Civil War with the Supreme Court’s infamous Dred Scott v. Sandford
decision in 1857, which denied citizenship to people of African descent even if they were not
enslaved. 
19


In those affirmations of the principle that anyone born in this country is 
automatically a citizen, race was the central issue — a fact that Trump and his advisers must
know. 
The order plainly violates the Constitution and seeks to overturn c

In [37]:
[hasattr(node, "id_") for node in documents]

[True]